# AI Toolkit by Ostris
## Solana Logo Single-Concept LoRA Training

This notebook trains a single-concept LoRA dedicated to the `cgl_sol` logo using `config/train_sol_lora.yaml`.


In [ ]:
!nvidia-smi


In [ ]:
import os
import subprocess

if os.path.exists('doom-ai-toolkit'):
    subprocess.run(['git', '-C', 'doom-ai-toolkit', 'pull', 'origin', 'main'], check=True)
    print('Repository updated successfully')
else:
    subprocess.run(['git', 'clone', 'https://github.com/posaune0423/doom-ai-toolkit.git'], check=True)
    print('Repository cloned successfully')


This notebook uses the dataset included in the repository as-is. The dataset for the Solana-like logo is already stored under `dataset/sol/`.


In [ ]:
!cd doom-ai-toolkit && git submodule update --init --recursive && pip install -r requirements.txt


In [ ]:
import os

!pip install --quiet --force-reinstall --no-deps numpy==1.26.3

os.kill(os.getpid(), 9)


## Model License
FLUX.1-schnell is licensed under Apache 2.0. You may need to agree to the model terms on Hugging Face and set `HF_TOKEN` if required.


In [ ]:
import os

SECRET_NAME = "HF_TOKEN"

from google.colab import userdata  # type: ignore

hf_token = userdata.get(SECRET_NAME)
if not hf_token:
    raise RuntimeError(
        f"Failed to retrieve required Hugging Face token from Colab secret '{SECRET_NAME}'."
    )

os.environ["HF_TOKEN"] = hf_token
print(f"HF_TOKEN environment variable has been set from Colab secret '{SECRET_NAME}'.")


In [ ]:
import os
import sys
sys.path.append('/content/doom-ai-toolkit')
from toolkit.job import run_job
import yaml
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"


## Setup

This cell loads the configuration file for the Solana logo LoRA, stored at `config/train_sol_lora.yaml`.


In [ ]:
from pathlib import Path

config_path = Path('/content/doom-ai-toolkit/config/train_sol_lora.yaml')
job_to_run = yaml.safe_load(config_path.read_text())
print('Configuration loaded')
print(f"Training: {job_to_run['config']['name']}")
print(f"Datasets: {len(job_to_run['config']['process'][0]['datasets'])}")


## Run it

Running `run_job` in the next cell will start training the Solana logo LoRA. Outputs will be saved under `/content/doom-ai-toolkit/output/sol_logo_lora` (the exact folder name depends on the run name).


In [ ]:
run_job(job_to_run)


## Done

After training finishes, you can retrieve the LoRA weights from the `output/` directory.


In [ ]:
from IPython.display import Image, display
import os
from pathlib import Path

base_output_dir = Path('/content/doom-ai-toolkit/output')

# Collect all "*/samples" directories under output and pick the most recently updated one
sample_dirs = [p for p in base_output_dir.glob('*/samples') if p.is_dir()]
if sample_dirs:
    samples_dir = sorted(sample_dirs, key=lambda p: p.stat().st_mtime)[-1]
    print(f'Using samples_dir: {samples_dir}')

    images = sorted([f for f in os.listdir(samples_dir) if f.endswith('.png')])
    for img in images[-6:]:
        display(Image(filename=os.path.join(samples_dir, img)))
else:
    print(f'No samples directory found under {base_output_dir}')
